In [1]:
import pandas as pd
from src.utils import generate_content_id

# Barter Deals


Goals:
- Construct the valid dataset for applications within 7 days
- See if I can obtain more predictors (deal value is an important one)

In [4]:
import os
from dotenv import load_dotenv
import numpy as np

import pandas as pd
from sqlalchemy import create_engine

# Load environment variables from .env file
load_dotenv("../private_data/.env")

host = os.getenv("HOST")
db   = os.getenv("DB")
port   = os.getenv("PORT")
role = os.getenv("ROLE")
pw   = os.getenv("PASSWORD")
engine = create_engine(f"postgresql+psycopg2://{role}:{pw}@{host}:{port}/{db}")


In [9]:
query = 'SELECT * FROM public.deals;'
df_deals = pd.read_sql(query, engine)
cols_of_interest = ['id', 'partner_id', 'creators_requirement', 'deal_value', 'social_requirement_type_id']

query = 'SELECT * FROM public.deals_computed;'
df_deals_comp = pd.read_sql(query, engine)

query = 'SELECT * FROM public.partners;'
df_partners = pd.read_sql(query, engine)

query = 'SELECT * FROM public.deal_social_requirement_types;'
df_social_requirement_types = pd.read_sql(query, engine)
dict_social_requirement_types = df_social_requirement_types.set_index('id')['min_followers'].to_dict()


In [156]:
df_deals_comp

,created_at,updated_at,deleted_at,id,deal_id,title,description,partner_id,locations,deal_type,...,last30_days_applicants_applications_count,last30_days_pending_applications_count,last30_days_upcoming_applications_count,last30_days_completed_applications_count,last30_days_cancelled_applications_count,accepts_international,accepted_countries,min_social_media_followers,live_since,live_until
0,2025-05-20 13:00:23.364737,2025-11-05 06:37:43.458654,NaT,3589,0196edc7-e6a8-00c8-6ff5-ea60fa3f2895,Paid deal: Moco Museum shoot (May 23 – €200),Moco Museum Amsterdam is inviting a small grou...,85,"[{'id': 194, 'name': 'Barter Agency', 'town': ...",physical,...,0,0,0,0,0,False,[NLD],2500,2025-05-20 15:21:28.249567,NaT
1,2025-05-01 05:38:46.147207,2025-05-27 06:47:49.284494,NaT,2803,019689e5-f489-00c8-8c26-06b3ea0961a5,Promotion clothing,Sharing unique and HQ content photos (one post...,153,None,online,...,0,0,0,0,0,False,[NLD],5000,2023-11-28 14:59:51.626780,NaT
2,2025-05-01 05:36:56.354872,2025-06-20 06:33:33.336563,NaT,622,019689e2-bfb5-00c8-6485-23352d64b0e5,Matrastopper,Wij zijn op zoek naar creatieve en grappige in...,1102,None,online,...,0,0,0,0,0,False,[NLD],5000,2025-03-04 11:46:18.176627,NaT
3,2025-05-01 05:37:43.926310,2025-08-11 19:54:20.702222,NaT,1527,019689e3-d7a5-00c8-f5a8-6bfe51a21cd6,FREE Oversized VIBE T-SHIRT from Vibecreation.nl,Vibecreation (VIBE) isn’t just a brand—it’s an...,1162,None,online,...,0,0,0,0,0,False,[NLD],2500,2025-04-17 09:50:45.518632,NaT
4,2025-05-01 05:38:37.126799,2025-11-05 06:31:29.851951,NaT,2614,019689e5-7dd0-00c8-615b-7750a8fec2f3,Ready-To-Blend Protein Smoothies targeted at c...,You will receive three Ready-To-Blend Smoothie...,1118,"[{'id': 1601, 'name': 'Val's Foods', 'town': '...",physical,...,0,0,0,0,0,False,[NLD],2500,2025-02-10 18:19:51.240203,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5106,2025-05-01 05:38:32.589051,2025-11-05 06:30:53.054004,NaT,2533,019689e5-4642-00c8-08dd-41d776586f1c,Private Museum Visit at Nxt Museum,Bring a friend and have the the entire museum ...,365,"[{'id': 444, 'name': 'Nxt Museum', 'town': 'No...",physical,...,0,0,0,0,0,False,[NLD],50000,2024-03-26 14:09:02.539962,NaT
5107,2025-05-01 05:38:35.681583,2025-11-05 06:31:18.352139,NaT,2587,019689e5-6ce0-00c8-5fdd-a058811d2946,Chocomates pictures,We are looking to promote our Chocomates produ...,661,"[{'id': 985, 'name': 'Mates Spirits BV', 'town...",physical,...,0,0,0,0,0,False,[NLD],10000,2024-10-17 14:59:27.484092,NaT
5108,2025-07-22 13:24:15.292976,2025-11-05 06:43:59.654823,NaT,4614,0198324e-635d-00c8-7d1d-bbb0b902b0d9,A night out with a chef's menu for two! (dupli...,Come and dine at De Japanner!\nWe'll provide a...,1790,"[{'id': 2642, 'name': 'De Japanner West', 'tow...",physical,...,0,0,0,0,0,True,"[NLD, GBR, BEL, DEU]",10000,NaT,NaT
5109,2025-07-24 13:21:26.536026,2025-11-05 06:44:13.856914,NaT,4659,01983c98-87ee-00c8-e551-8fd9afde13aa,Comedy Club (duplicate),Op woensdag is het tijd voor een comedy avond ...,967,"[{'id': 1452, 'name': 'Holy Moly', 'town': 'No...",physical,...,0,0,0,0,0,False,[NLD],5000,NaT,NaT


In [157]:
df = pd.merge(df_deals_comp[['applicants_applications_count', 'content_types', 'deal_id', 'main_image', 'min_social_media_followers', 'deal_tags', 'live_since']], df_deals, left_on = 'deal_id', right_on = 'id')

## EDA

In [10]:
df_deals_comp

,created_at,updated_at,deleted_at,id,deal_id,title,description,partner_id,locations,deal_type,...,last30_days_applicants_applications_count,last30_days_pending_applications_count,last30_days_upcoming_applications_count,last30_days_completed_applications_count,last30_days_cancelled_applications_count,accepts_international,accepted_countries,min_social_media_followers,live_since,live_until
0,2025-05-20 13:00:23.364737,2025-11-05 06:37:43.458654,NaT,3589,0196edc7-e6a8-00c8-6ff5-ea60fa3f2895,Paid deal: Moco Museum shoot (May 23 – €200),Moco Museum Amsterdam is inviting a small grou...,85,"[{'id': 194, 'name': 'Barter Agency', 'town': ...",physical,...,0,0,0,0,0,False,[NLD],2500,2025-05-20 15:21:28.249567,NaT
1,2025-05-01 05:38:46.147207,2025-05-27 06:47:49.284494,NaT,2803,019689e5-f489-00c8-8c26-06b3ea0961a5,Promotion clothing,Sharing unique and HQ content photos (one post...,153,None,online,...,0,0,0,0,0,False,[NLD],5000,2023-11-28 14:59:51.626780,NaT
2,2025-05-01 05:36:56.354872,2025-06-20 06:33:33.336563,NaT,622,019689e2-bfb5-00c8-6485-23352d64b0e5,Matrastopper,Wij zijn op zoek naar creatieve en grappige in...,1102,None,online,...,0,0,0,0,0,False,[NLD],5000,2025-03-04 11:46:18.176627,NaT
3,2025-05-01 05:37:43.926310,2025-08-11 19:54:20.702222,NaT,1527,019689e3-d7a5-00c8-f5a8-6bfe51a21cd6,FREE Oversized VIBE T-SHIRT from Vibecreation.nl,Vibecreation (VIBE) isn’t just a brand—it’s an...,1162,None,online,...,0,0,0,0,0,False,[NLD],2500,2025-04-17 09:50:45.518632,NaT
4,2025-05-01 05:38:37.126799,2025-11-05 06:31:29.851951,NaT,2614,019689e5-7dd0-00c8-615b-7750a8fec2f3,Ready-To-Blend Protein Smoothies targeted at c...,You will receive three Ready-To-Blend Smoothie...,1118,"[{'id': 1601, 'name': 'Val's Foods', 'town': '...",physical,...,0,0,0,0,0,False,[NLD],2500,2025-02-10 18:19:51.240203,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5106,2025-05-01 05:38:32.589051,2025-11-05 06:30:53.054004,NaT,2533,019689e5-4642-00c8-08dd-41d776586f1c,Private Museum Visit at Nxt Museum,Bring a friend and have the the entire museum ...,365,"[{'id': 444, 'name': 'Nxt Museum', 'town': 'No...",physical,...,0,0,0,0,0,False,[NLD],50000,2024-03-26 14:09:02.539962,NaT
5107,2025-05-01 05:38:35.681583,2025-11-05 06:31:18.352139,NaT,2587,019689e5-6ce0-00c8-5fdd-a058811d2946,Chocomates pictures,We are looking to promote our Chocomates produ...,661,"[{'id': 985, 'name': 'Mates Spirits BV', 'town...",physical,...,0,0,0,0,0,False,[NLD],10000,2024-10-17 14:59:27.484092,NaT
5108,2025-07-22 13:24:15.292976,2025-11-05 06:43:59.654823,NaT,4614,0198324e-635d-00c8-7d1d-bbb0b902b0d9,A night out with a chef's menu for two! (dupli...,Come and dine at De Japanner!\nWe'll provide a...,1790,"[{'id': 2642, 'name': 'De Japanner West', 'tow...",physical,...,0,0,0,0,0,True,"[NLD, GBR, BEL, DEU]",10000,NaT,NaT
5109,2025-07-24 13:21:26.536026,2025-11-05 06:44:13.856914,NaT,4659,01983c98-87ee-00c8-e551-8fd9afde13aa,Comedy Club (duplicate),Op woensdag is het tijd voor een comedy avond ...,967,"[{'id': 1452, 'name': 'Holy Moly', 'town': 'No...",physical,...,0,0,0,0,0,False,[NLD],5000,NaT,NaT


In [41]:
dict_social_requirement_types

{3: 5000,
 4: 10000,
 5: 25000,
 6: 50000,
 2: 2500,
 1: 1500,
 9: 200000,
 7: 100000,
 8: 125000,
 34: 0}

In [65]:
df = df.dropna(subset = ['social_requirement_type_id'])

In [57]:
df

,applicants_applications_count,content_types,deal_id,main_image,min_social_media_followers,deal_tags,created_at,updated_at,deleted_at,id,...,accepted_countries,social_requirement_type_id,product_name,schedule_type,schedule_model,legacy_id,tags,gender,featured_image,company_id
0,39,"[{'id': 41, 'name': 'Activities', 'slug': 'per...",0196edc7-e6a8-00c8-6ff5-ea60fa3f2895,uploads/deals/0196edc7-5969-ffff-6700-7ae699b8...,2500,None,2025-05-20 13:00:23.080212,2025-10-28 10:55:34.719187,NaT,0196edc7-e6a8-00c8-6ff5-ea60fa3f2895,...,[NLD],2.0,,physical_every_day_all_day_schedule,"{'indefinitely': True, 'date_schedule_end_at':...",NaN,None,None,None,0199f1d4-b501-015e-5c45-1bc8300b3949
1,0,"[{'id': 21, 'name': 'Fashion', 'slug': 'Dress'}]",019689e5-f489-00c8-8c26-06b3ea0961a5,uploads/deals/019689e5-f54b-ffff-dbed-637e9d2a...,5000,None,2023-11-28 14:59:51.626780,2025-05-01 03:31:11.600095,NaT,019689e5-f489-00c8-8c26-06b3ea0961a5,...,[NLD],3.0,,online_indefinitely_schedule,{},487.0,None,None,None,None
2,23,"[{'id': 29, 'name': 'UGC', 'slug': 'Film-Strip'}]",019689e2-bfb5-00c8-6485-23352d64b0e5,uploads/deals/019689e2-c06c-ffff-baf0-c7e9decd...,5000,None,2025-03-04 11:46:18.176627,2025-05-01 03:27:41.316646,NaT,019689e2-bfb5-00c8-6485-23352d64b0e5,...,[NLD],3.0,,online_indefinitely_schedule,{},3593.0,None,None,None,None
3,43,"[{'id': 21, 'name': 'Fashion', 'slug': 'Dress'}]",019689e3-d7a5-00c8-f5a8-6bfe51a21cd6,uploads/deals/019689e3-d86f-ffff-a03e-d38765a0...,2500,None,2025-04-17 09:50:45.518632,2025-06-08 09:05:21.413273,NaT,019689e3-d7a5-00c8-f5a8-6bfe51a21cd6,...,[NLD],2.0,,online_indefinitely_schedule,{},4228.0,None,None,None,None
4,13,"[{'id': 19, 'name': 'Sport', 'slug': 'Soccer-B...",019689e5-7dd0-00c8-615b-7750a8fec2f3,uploads/deals/019689e5-7f9d-ffff-5b6d-a335c1ce...,2500,None,2025-02-10 18:19:51.240203,2025-10-28 10:51:05.147656,NaT,019689e5-7dd0-00c8-615b-7750a8fec2f3,...,[NLD],2.0,,physical_every_day_all_day_schedule,"{'indefinitely': True, 'date_schedule_end_at':...",3242.0,None,None,None,0199f1b0-2b6c-015e-e5b3-6b1dd7564d3a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5106,9,"[{'id': 39, 'name': 'Experiences', 'slug': 'Te...",019689e5-4642-00c8-08dd-41d776586f1c,uploads/deals/019689e5-46b6-ffff-2159-fb70a484...,50000,None,2024-03-26 14:09:02.539962,2025-10-28 10:54:41.617599,NaT,019689e5-4642-00c8-08dd-41d776586f1c,...,[NLD],6.0,,physical_specific_days_all_day_schedule,"{'indefinitely': True, 'date_schedule_end_at':...",865.0,None,None,None,0199f1cc-e3f7-015e-432a-d2035c7ba015
5107,11,"[{'id': 39, 'name': 'Experiences', 'slug': 'Te...",019689e5-6ce0-00c8-5fdd-a058811d2946,uploads/deals/019689e5-6e0d-ffff-4a2f-362bb635...,10000,None,2024-10-17 14:59:27.484092,2025-10-28 10:52:42.692581,NaT,019689e5-6ce0-00c8-5fdd-a058811d2946,...,[NLD],4.0,,physical_every_day_all_day_schedule,"{'indefinitely': True, 'date_schedule_end_at':...",1889.0,None,None,None,0199f1c3-fcaf-015e-eeb9-cc244704412b
5108,0,"[{'id': 23, 'name': 'Food', 'slug': 'Pizza'}]",0198324e-635d-00c8-7d1d-bbb0b902b0d9,,10000,None,2025-07-22 13:24:14.813323,2025-10-28 11:04:12.121141,NaT,0198324e-635d-00c8-7d1d-bbb0b902b0d9,...,"[NLD, GBR, BEL, DEU]",4.0,https://www.dejapanner.com/,physical_specific_days_specific_time_schedule,{'end_time': {'RFC3339': '2025-06-22T19:00:00Z...,NaN,None,None,None,0199f18e-af17-015e-23aa-c780e5c4a2b8
5109,0,"[{'id': 41, 'name': 'Activities', 'slug': 'per...",01983c98-87ee-00c8-e551-8fd9afde13aa,,5000,None,2025-07-24 13:21:25.998303,2025-10-28 10:51:32.951354,NaT,01983c98-87ee-00c8-e551-8fd9afde13aa,...,[NLD],3.0,https://holymolybreda.nl/comedy-club/,physical_specific_days_specific_time_schedule,{'end_time': {'RFC3339': '2025-06-24T20:30:00Z...,NaN,None,None,None,0199f1b6-5645-015e-ffbf-5e1786a1e5c3


In 25 rows, the min social media followers does not correspond to the social requirement type ID. These are very few rows, and likely do not affect downstream analysis. I will continue with the min_social_media_followers column. 

In [ ]:
df['inferred_min_social_media_followers'] = df['social_requirement_type_id'].map(dict_social_requirement_types)
df[df['inferred_min_social_media_followers'] != df['min_social_media_followers']]


25\% of deals have a deal value. This can be valuable information, although at first sight there does not seem to be a correlation between deal value and the number of applicants. We will see in modelling whether this is still correct when adding other variables. 

In [7]:
sum(df_deals.deal_value > 0) / len(df_deals)

0.2545763351575769

## Applications within $n$ days after going live

In [5]:
query = 'SELECT * FROM public.deal_activity_logs;'
df_deals_log = pd.read_sql(query, engine)

query = 'SELECT * FROM public.deal_applications;'
df_applications = pd.read_sql(query, engine)

In [7]:
df_deals_log

,created_at,updated_at,deleted_at,id,entity_type,entity_id,action,before,after,metadata
0,2025-06-14 04:00:00.320670,2025-07-11 14:40:32.801163,None,01976c98-2740-00cd-fe8b-f9c564ae29ac,deal,0196b417-9214-00c8-c0c6-9963c186bbe9,DEAL_UPDATE,"{'deal': {'SoftDeletable': {}, 'accepted_count...","{'deal': {'SoftDeletable': {}, 'accepted_count...",{'entity_global_id': '0196b417-9214-00c8-c0c6-...
1,2025-06-14 02:15:58.418330,2025-07-11 14:40:32.804326,None,01976c38-e8d2-00cd-879b-178e48af5364,deal_application,01976c38-e890-00c9-80a7-81b8b10c16aa,DEAL_APPLICATION_CREATED,"{'application': {'SoftDeletable': {}, 'cancell...","{'application': {'SoftDeletable': {}, 'cancell...",{'entity_global_id': '019689e2-fbdf-00c8-d5c1-...
2,2025-06-14 02:13:49.773612,2025-07-11 14:40:32.807938,None,01976c36-f24d-00cd-24a6-88f8f08e06c6,deal_application,01976c36-f202-00c9-5479-c9952c80441a,DEAL_APPLICATION_CREATED,"{'application': {'SoftDeletable': {}, 'cancell...","{'application': {'SoftDeletable': {}, 'cancell...",{'entity_global_id': '0196f838-48aa-00c8-9a89-...
3,2025-06-14 01:04:27.294186,2025-07-11 14:40:32.811161,None,01976bf7-6e9d-00cd-b751-6b65c62a8130,deal_application,01976bf7-6e4e-00c9-0a9e-b972e4f37bdb,DEAL_APPLICATION_CREATED,"{'application': {'SoftDeletable': {}, 'cancell...","{'application': {'SoftDeletable': {}, 'cancell...",{'entity_global_id': '019689e3-e641-00c8-1f55-...
4,2025-06-14 00:26:58.841062,2025-07-11 14:40:32.813614,None,01976bd5-1f99-00cd-5bb1-832b23e07cb4,deal_application,01976bd5-1f85-00c9-aca4-f92e30696a43,DEAL_APPLICATION_CREATED,"{'application': {'SoftDeletable': {}, 'cancell...","{'application': {'SoftDeletable': {}, 'cancell...",{'entity_global_id': '0196f7d8-8767-00c8-238a-...
...,...,...,...,...,...,...,...,...,...,...
81702,2025-11-07 14:57:25.522140,2025-11-07 14:57:25.522140,None,019a5ed2-8212-00cd-7a28-aa9b4ddded5f,deal_application,019a5e99-9b72-00c9-3db1-51df4560b168,DEAL_APPLICATION_CANCELLED,{'application': {'SoftDeletable': {'created_at...,{'application': {'SoftDeletable': {'created_at...,{'entity_global_id': '0198f6dd-19a7-00c8-c3df-...
81703,2025-11-07 15:02:08.291930,2025-11-07 15:02:08.291930,None,019a5ed6-d2a3-00cd-c51b-7f414d947368,deal_application,019a5ed6-d211-00c9-4c87-af10996a25fa,DEAL_APPLICATION_CREATED,{'version': 1},{'application': {'SoftDeletable': {'created_at...,{'entity_global_id': '019961b8-2bf8-00c8-68fe-...
81704,2025-11-07 15:02:03.866244,2025-11-07 15:02:03.866244,None,019a5ed6-c159-00cd-251a-26144436b984,deal_application,019a5ed6-c0a1-00c9-3a94-3f232e3730ae,DEAL_APPLICATION_CREATED,{'version': 1},{'application': {'SoftDeletable': {'created_at...,{'entity_global_id': '0198f6dd-19a7-00c8-c3df-...
81705,2025-11-07 15:02:06.030463,2025-11-07 15:02:06.030463,None,019a5ed6-c9ce-00cd-308b-2733306b30aa,deal_application,019a5ed6-c9a0-00c9-174c-7ab04332d5b4,DEAL_APPLICATION_CREATED,{'version': 1},{'application': {'SoftDeletable': {'created_at...,{'entity_global_id': '0198f665-cc50-00c8-1c06-...


In [6]:
df_applications

,id,partner_id,location_id,deal_id,creator_id,rescheduling_deal_application_id,schedule_date,status,cancelled_at,cancelled_by,cancelled_by_user_id,cancellation_reason,created_at,updated_at,deleted_at,legacy_id,cancellation_remarks
0,01968a03-c147-00c9-fdc6-af5f46165ec3,424,566.0,019689e3-42ab-00c8-3759-54e881975755,238,None,2024-07-11 00:00:00+00:00,cancelled,NaT,None,NaN,None,2024-06-24 14:04:19.999753,2025-10-28 11:42:56.873559,NaT,46577.0,[other]
1,01968a03-c1ad-00c9-4ae0-207614d26b70,424,566.0,019689e3-42ab-00c8-3759-54e881975755,40,None,2024-08-01 00:00:00+00:00,cancelled,NaT,None,NaN,None,2024-07-04 23:19:38.937334,2025-10-28 11:42:56.873559,NaT,48394.0,[other]
2,01968a03-b0e2-00c9-ac5d-dd8728a255e8,424,566.0,019689e3-42ab-00c8-3759-54e881975755,752,None,2024-09-12 00:00:00+00:00,cancelled,NaT,None,NaN,None,2024-08-14 06:14:17.712206,2025-10-28 11:42:56.873559,NaT,54466.0,[other]
3,01968a23-274b-00c9-8c98-85b4c81bd2fb,58,183.0,019689e4-9d23-00c8-77b0-21e7393b0ae2,991,None,2024-04-04 00:00:00+00:00,rejected,NaT,None,NaN,None,2024-03-26 22:11:06.671629,2025-10-28 11:43:41.718447,NaT,31590.0,[other]
4,01968a03-bef4-00c9-76a5-30f4bc480c0b,424,566.0,019689e3-42ab-00c8-3759-54e881975755,9419,None,2024-07-11 00:00:00+00:00,cancelled,NaT,None,NaN,None,2024-06-02 13:09:32.573065,2025-10-28 11:42:56.873559,NaT,42546.0,[other]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195417,01968a19-8746-00c9-acb6-b8a8a2a51801,132,184.0,019689e5-a977-00c8-8b48-95f0e6948da7,370,None,2023-12-06 00:00:00+00:00,cancelled,NaT,None,NaN,None,2023-11-26 23:22:55.204678,2025-10-28 11:43:41.549869,NaT,6567.0,[other]
195418,01968a19-8762-00c9-cb6f-c94262b8c832,132,184.0,019689e5-a977-00c8-8b48-95f0e6948da7,200,None,2023-12-07 00:00:00+00:00,cancelled,NaT,None,NaN,None,2023-12-04 14:04:20.228504,2025-10-28 11:43:41.549869,NaT,8502.0,[other]
195419,01968a19-87b1-00c9-5586-5e8dab54baba,132,184.0,019689e5-a977-00c8-8b48-95f0e6948da7,2004,None,2024-01-03 00:00:00+00:00,cancelled,NaT,None,NaN,None,2023-12-18 22:21:07.473382,2025-10-28 11:43:41.549869,NaT,11356.0,[other]
195420,01968a19-870b-00c9-0cc0-78ba48baefbf,132,184.0,019689e5-a977-00c8-8b48-95f0e6948da7,487,None,2024-02-13 00:00:00+00:00,cancelled,NaT,None,NaN,None,2024-02-05 13:39:01.976287,2025-10-28 11:43:41.549869,NaT,20099.0,[other]


I want to compute the number of applications within 7 days after the deal has gone live. Problematically, not each row corresponds to a unique application. The idea of simply counting the rows in the first 7 days after going live is invalid. To obtain valid data, I need to know which rows I can safely use. 

I have zeroed in on the problem. A database migration has led to duplicate rows. My analysis shows that the rows where the 'deleted_at' column is NA are valid. 

To this end, when calculating the applications within $n$ days of going live, we first determine whether there is a data quality problem for that specific deal, by checking whether the number of rows in the applications log for that deal corresponds to the number of applications. If it does not, I only consider the rows where 'deleted_at' is NA. 

In [ ]:
def apps_after_n_days_test(row):
    subs = df_applications[df_applications['deal_id'] == row['deal_id']]
    if len(subs) != row['applicants_applications_count']:
        return sum(subs['deleted_at'].isna())
        # return 'banaan'
    else:
        return len(subs)

In [ ]:
a = df.apply(apps_after_n_days_test, axis = 1)
df['test_nr_of_apps'] = a
df['diff'] = a - df['applicants_applications_count']
df['diff'].value_counts()

In [ ]:
def apps_after_n_days(row, n=7):
    subs = df_applications[df_applications['deal_id'] == row['deal_id']]
    if len(subs) != row['applicants_applications_count']:
        subs = subs[subs['deleted_at'].isna()]

    # "created_at" is the date on which the application log entry was created (application was made)
    days_since_live = (subs.created_at - row.live_since).dt.days
    return ((days_since_live >= 0) & (days_since_live <= n)).sum()

In [295]:
apps_after_7_days = df.apply(apps_after_n_days, axis=1)

In [297]:
df['apps_after_7_days'] = apps_after_7_days

In [319]:
old_df = pd.read_parquet('../data/raw/BARTER_DEALS_OLD.parquet')
df['apps_after_7_days_OLD'] = old_df['applications_till_day_7']

In [320]:
df['deal_id'] = df['deal_id'].astype(str)
df['id'] = df['id'].astype(str)
df['company_id'] = df['company_id'].astype(str)

In [321]:
df.to_parquet('../data/raw/BARTER_DEALS.parquet')

Need to do some manual column renaming

In [ ]:
# import yaml
# from src import paths
# import pandas as pd

# with open('../src/configs/config.yaml', 'r') as f:
#     full_config = yaml.safe_load(f)

# raw_df = pd.read_parquet(f"{paths.RAW_DATA_DIR}/{full_config['active_analysis'].upper()}.parquet")

# raw_df = raw_df.rename(columns = {'title': 'deal_title', 'description': 'deal_text'})
# raw_df.to_parquet('../data/raw/BARTER_DEALS.parquet')

------------

Comparison with old dataset

In [312]:
old_df = pd.read_parquet('../data/raw/BARTER_DEALS_OLD.parquet')
new_df = pd.read_parquet('../data/raw/BARTER_DEALS.parquet') 

In [324]:
df['diff'] = new_df['apps_after_7_days'] - old_df['applications_till_day_7']

In [318]:
new_df[(new_df['apps_after_7_days'] - old_df['applications_till_day_7']) != 0]

,applicants_applications_count,content_types,deal_id,main_image,min_social_media_followers,deal_tags,live_since,created_at,updated_at,deleted_at,...,schedule_model,legacy_id,tags,gender,featured_image,company_id,test_nr_of_apps,update_at_date,diff,apps_after_7_days
10,5,"[{'id': 23, 'name': 'Food', 'slug': 'Pizza'}]",019689e4-1ac5-00c8-aac6-bcb8c2a3b55a,uploads/deals/019689e4-1be9-ffff-37b8-4090d6f1...,5000,None,2025-04-18 14:10:58.460706,2025-04-18 14:10:58.460706,2025-10-28 10:49:45.450337,NaT,...,"{'date_schedule_end_at': {'day': 30.0, 'hours'...",4264.0,None,None,None,0199f19e-c73d-015e-76b1-67c22e995a05,5,2025-10-28,0,4
12,0,"[{'id': 41, 'name': 'Activities', 'slug': 'per...",01985615-068a-00c8-502e-689ad4f46f4f,uploads/deals/01985615-0778-ffff-5f39-475ebacc...,1500,None,NaT,2025-07-29 12:07:55.274571,2025-07-29 12:08:05.638372,NaT,...,"{'date_schedule_end_at': None, 'date_schedule_...",NaN,None,None,None,None,0,2025-07-29,0,0
20,0,"[{'id': 21, 'name': 'Fashion', 'slug': 'Dress'}]",0196cf30-3d9e-00c8-45de-a6091919cc67,,1500,None,NaT,2025-05-14 14:26:07.390204,2025-05-14 14:38:34.607315,2025-05-15 09:33:30.588079+00:00,...,"{'date_schedule_end_at': None, 'date_schedule_...",NaN,None,None,None,None,0,2025-05-14,0,0
22,0,"[{'id': 41, 'name': 'Activities', 'slug': 'per...",019689e2-4ed5-00c8-4120-a08641f9f6f7,uploads/deals/019689e2-4ff4-ffff-5f8b-033c32b8...,5000,None,2024-05-02 09:27:34.909380,2024-05-02 09:27:34.909380,2025-05-01 03:27:12.709921,NaT,...,"{'date_schedule_end_at': None, 'date_schedule_...",1068.0,None,None,None,None,0,2025-05-01,0,0
23,0,"[{'id': 21, 'name': 'Fashion', 'slug': 'Dress'}]",0196cf35-c5d6-00c8-4d97-bbfef60b2cf5,,2500,None,NaT,2025-05-14 14:32:09.942486,2025-05-14 14:32:15.868080,2025-05-14 14:35:11.325650+00:00,...,"{'date_schedule_end_at': None, 'date_schedule_...",NaN,None,None,None,None,0,2025-05-14,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5080,18,"[{'id': 41, 'name': 'Activities', 'slug': 'per...",01984634-66d5-00c8-ad93-d24a2702fa3b,uploads/deals/01984633-f55f-ffff-2cf1-8675b143...,5000,None,2025-07-26 10:08:16.271592,2025-07-26 10:08:16.085273,2025-07-26 10:08:16.271633,NaT,...,"{'date_schedule_end_at': None, 'date_schedule_...",NaN,None,None,None,None,18,2025-07-26,0,10
5081,11,"[{'id': 41, 'name': 'Activities', 'slug': 'per...",019754be-ceef-00c8-5b09-dfed9e1a4d5b,uploads/deals/019754da-c311-ffff-ea18-37220ffe...,10000,None,2025-06-09 13:50:11.536643,2025-06-09 12:51:20.431174,2025-10-28 10:49:05.541885,NaT,...,"{'date_schedule_end_at': {'day': 0.0, 'hours':...",NaN,None,None,None,0199f191-1fd6-015e-5b34-b4c261ecec51,11,2025-10-28,0,3
5089,18,"[{'id': 25, 'name': 'Cooking', 'slug': 'Cookin...",019689e4-a17f-00c8-ef97-22b05ba8c294,uploads/deals/019689e4-a278-ffff-d115-68616602...,5000,None,2025-03-24 15:55:44.036700,2025-03-24 15:55:44.036700,2025-10-28 10:50:25.064695,NaT,...,"{'date_schedule_end_at': {'day': 13.0, 'hours'...",3899.0,None,None,None,0199f1a5-4b71-015e-9f6f-0ca21de5dc39,18,2025-10-28,0,11
5108,0,"[{'id': 23, 'name': 'Food', 'slug': 'Pizza'}]",0198324e-635d-00c8-7d1d-bbb0b902b0d9,,10000,None,NaT,2025-07-22 13:24:14.813323,2025-10-28 11:04:12.121141,NaT,...,"{'date_schedule_end_at': {'day': 0.0, 'hours':...",NaN,None,None,None,0199f18e-af17-015e-23aa-c780e5c4a2b8,0,2025-10-28,0,0


In [313]:
old_df['applications_till_day_7']

0       39
1        0
2       13
3       19
4       13
        ..
5104    26
5105     7
5106     5
5107     7
5110    10
Name: applications_till_day_7, Length: 4203, dtype: int64

-------

Testing

In [213]:
subs = df[df['test_nr_of_apps'] == 'banaan']

In [214]:
subs['updated_at_date'] = subs['updated_at'].apply(lambda x: x.date())

/var/folders/j6/_nwsrmhx5w1fj_nrxmrss8180000gq/T/ipykernel_86023/599127219.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subs['updated_at_date'] = subs['updated_at'].apply(lambda x: x.date())


In [224]:
df_applications['updated_at_date'] = df_applications['updated_at'].apply(lambda x: x.date())

In [ ]:
def apply_value_counts(row):
    fn_subs = df_applications[df_applications['deal_id'] == row['deal_id']]
    try:
        return fn_subs.updated_at_date.value_counts().iloc[0]
    except:
        return 'poep'

In [282]:
def apply_value_counts(row):
    fn_subs = df_applications[df_applications['deal_id'] == row['deal_id']]
    try:
        return sum(fn_subs.deleted_at.isna())
    except:
        return 'poep'

I have locked in the problem to the two dates below. If I sum the value counts where the index corresponds to these two dates, then I should obtain the nr of applications.

In [283]:
a = subs.apply(apply_value_counts, axis = 1)

In [284]:
subs['updated_at_value_counts'] = a
subs['diff'] = a - subs['applicants_applications_count']
subs_sorted = subs.sort_values('diff')

/var/folders/j6/_nwsrmhx5w1fj_nrxmrss8180000gq/T/ipykernel_86023/407119917.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subs['updated_at_value_counts'] = a
/var/folders/j6/_nwsrmhx5w1fj_nrxmrss8180000gq/T/ipykernel_86023/407119917.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subs['diff'] = a - subs['applicants_applications_count']


In [208]:
subs_sorted.deal_id.iloc[0]

UUID('019689e4-da92-00c8-8339-34136a24f15a')

In [281]:
test = df_applications[df_applications.deal_id == subs_sorted.deal_id.iloc[0]]

In [ ]:
test

In [277]:
a = test.apply(lambda x: x['cancellation_remarks'][0] == 'other', axis = 1)

In [280]:
test[~a]

,id,partner_id,location_id,deal_id,creator_id,rescheduling_deal_application_id,schedule_date,status,cancelled_at,cancelled_by,cancelled_by_user_id,cancellation_reason,created_at,updated_at,deleted_at,legacy_id,cancellation_remarks,updated_at_date
3504,0196ab5b-b136-00c9-5f1d-bfc3d32f9f65,1520,NaN,019689e4-0154-00c8-06bf-3126846dceff,28551,None,NaT,rejected,2025-05-18 20:20:23.797072+00:00,partner,41022.0,,2025-05-07 15:27:15.254072,2025-05-18 20:20:23.797113,NaT,NaN,[not_a_match],2025-05-18
3586,0197c6b7-6318-00c9-b0ab-90d6de7190ad,1520,NaN,019689e4-0154-00c8-06bf-3126846dceff,30673,None,NaT,rejected,2025-07-16 15:17:41.485801+00:00,partner,41022.0,,2025-07-01 15:59:56.696502,2025-07-16 15:17:41.485847,NaT,NaN,[not_a_match],2025-07-16
3807,01970823-ed87-00c9-2e22-779dfb5b1367,1520,NaN,019689e4-0154-00c8-06bf-3126846dceff,7618,None,NaT,rejected,2025-05-28 07:35:18.070853+00:00,partner,41022.0,,2025-05-25 15:51:01.767937,2025-05-28 07:35:18.070906,NaT,NaN,[not_a_match],2025-05-28
4974,0196abb2-a364-00c9-b11c-edb1041975f6,1520,NaN,019689e4-0154-00c8-06bf-3126846dceff,1046,None,NaT,rejected,2025-05-13 07:16:01.425317+00:00,partner,41022.0,,2025-05-07 17:02:13.348343,2025-05-13 07:16:01.425368,NaT,NaN,[not_a_match],2025-05-13
16462,01979ec5-73bc-00c9-54ae-3df3655d17f2,1520,NaN,019689e4-0154-00c8-06bf-3126846dceff,29362,None,NaT,rejected,2025-07-16 15:18:54.590727+00:00,partner,41022.0,,2025-06-23 21:50:29.820543,2025-07-16 15:18:54.590773,NaT,NaN,[not_a_match],2025-07-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153781,0198569e-abca-00c9-5cd9-b4b69202e161,1520,NaN,019689e4-0154-00c8-06bf-3126846dceff,30861,None,NaT,rejected,2025-08-05 09:33:33.718866+00:00,partner,41022.0,,2025-07-29 14:38:16.010132,2025-08-05 09:33:33.718973,NaT,NaN,[not_a_match],2025-08-05
154040,01985db8-c2d7-00c9-7d66-f7fdcc92bb89,1520,NaN,019689e4-0154-00c8-06bf-3126846dceff,2999,None,NaT,rejected,2025-08-04 21:09:02.328662+00:00,partner,41022.0,,2025-07-30 23:44:06.360038,2025-08-04 21:09:02.328744,NaT,NaN,[not_a_match],2025-08-04
154166,01985bca-9592-00c9-02db-ed5bb6ddeba1,1520,NaN,019689e4-0154-00c8-06bf-3126846dceff,21350,None,NaT,rejected,2025-08-05 09:35:08.969635+00:00,partner,41022.0,,2025-07-30 14:44:19.986154,2025-08-05 09:35:08.969678,NaT,NaN,[not_a_match],2025-08-05
155252,01986b48-a54d-00c9-88ca-fe4ce102d65c,1520,NaN,019689e4-0154-00c8-06bf-3126846dceff,5190,None,NaT,rejected,2025-08-05 09:34:45.300209+00:00,partner,41022.0,,2025-08-02 14:56:19.789961,2025-08-05 09:34:45.300297,NaT,NaN,[not_a_match],2025-08-05


In [275]:
test['cancellation_remarks'].iloc[0][0]

'not_a_match'

In [266]:
test.updated_at_date.value_counts()[:7]

updated_at_date
2025-05-01    44
2025-08-05    31
2025-07-16    30
2025-06-21    21
2025-05-20    14
2025-06-08    10
2025-05-29     9
Name: count, dtype: int64

In [254]:
co = test.updated_at_date.value_counts()

189

In [ ]:
subs

When we check by legacy id, only a single row shows up in the applications log. If I want to use the applications log to get a valid measurement for applications_in_first_7_days, I need to verify whether I can obtain correct data on this deal or not. 

I observe that the legacy_id is from applications log is not a mapping to legacy_id in the deals table. 

In [194]:
hotspot_id = df_applications[df_applications.legacy_id == 1098]['deal_id'].iloc[0]

In [195]:
df_applications[df_applications['deal_id'] == hotspot_id]

,id,partner_id,location_id,deal_id,creator_id,rescheduling_deal_application_id,schedule_date,status,cancelled_at,cancelled_by,cancelled_by_user_id,cancellation_reason,created_at,updated_at,deleted_at,legacy_id,cancellation_remarks
29279,01968a04-6c64-00c9-71b0-f9b46bcdb01c,49,60.0,019689e3-2e4b-00c8-6f12-a6e09e73f34c,657,None,2023-10-25 00:00:00+00:00,finished,NaT,None,NaN,None,2023-10-24 09:14:57.326915,2025-10-28 11:43:58.257343,NaT,3077.0,[other]
44625,01968a04-4d32-00c9-e555-787c712c6989,49,60.0,019689e3-2e4b-00c8-6f12-a6e09e73f34c,1227,None,2023-11-24 00:00:00+00:00,finished,NaT,None,NaN,None,2023-11-22 16:57:28.012884,2025-10-28 11:43:58.257343,NaT,6319.0,[other]
45237,01968a04-5321-00c9-77de-4ad398feebd2,49,60.0,019689e3-2e4b-00c8-6f12-a6e09e73f34c,525,None,2023-11-02 00:00:00+00:00,finished,NaT,None,NaN,None,2023-10-31 15:00:33.498023,2025-10-28 11:43:58.257343,NaT,3796.0,[other]
45315,01968a04-6dae-00c9-362a-ae2db605c9ed,49,60.0,019689e3-2e4b-00c8-6f12-a6e09e73f34c,688,None,2023-10-27 00:00:00+00:00,finished,NaT,None,NaN,None,2023-10-25 11:39:24.413232,2025-10-28 11:43:58.257343,NaT,3256.0,[other]
45430,01968a04-7195-00c9-b596-051606628d58,49,60.0,019689e3-2e4b-00c8-6f12-a6e09e73f34c,593,None,2023-10-25 00:00:00+00:00,finished,NaT,None,NaN,None,2023-10-20 22:11:16.493153,2025-10-28 11:43:58.257343,NaT,2905.0,[other]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75869,01968a04-a003-00c9-863f-a075c31b6371,49,60.0,019689e3-2e4b-00c8-6f12-a6e09e73f34c,199,None,2023-10-12 00:00:00+00:00,cancelled,NaT,None,NaN,None,2023-10-03 10:19:40.068595,2025-10-28 11:43:58.257343,NaT,1608.0,[other]
75870,01968a04-a254-00c9-dda9-7b852bba2e44,49,60.0,019689e3-2e4b-00c8-6f12-a6e09e73f34c,414,None,2023-11-03 00:00:00+00:00,cancelled,NaT,None,NaN,None,2023-11-03 14:36:49.203587,2025-10-28 11:43:58.257343,NaT,4073.0,[other]
75871,01968a04-a272-00c9-9ed6-b6bb68ae0c50,49,60.0,019689e3-2e4b-00c8-6f12-a6e09e73f34c,414,None,2023-11-04 00:00:00+00:00,cancelled,NaT,None,NaN,None,2023-11-03 14:36:49.206867,2025-10-28 11:43:58.257343,NaT,4074.0,[other]
75872,01968a04-8521-00c9-495a-f9ed96c6c03a,49,60.0,019689e3-2e4b-00c8-6f12-a6e09e73f34c,1884,None,2023-12-21 00:00:00+00:00,cancelled,NaT,None,NaN,None,2023-11-29 15:56:45.456744,2025-10-28 11:43:58.257343,NaT,7217.0,[other]


In [174]:
df['update_at_date'] = df['updated_at'].apply(lambda x: x.date())


In [ ]:
def nrow_in_deal_applications_table(row):
    subs = df_applications[df_applications['deal_id'] == row['deal_id']]

    # return subs.updated_at.value_counts().iloc[0]
    return sum(subs['status'] != 'cancelled')
    # return len(subs)

In [126]:
a = df.apply(nrow_in_deal_applications_table, axis = 1)

IndexError: single positional indexer is out-of-bounds

In [104]:
df['nrow_in_deal_applications_table_without_cancelled_status'] = a

In [106]:
more_rows_than_apps = df[df['applicants_applications_count'] < df['nrow_in_deal_applications_table']]

Consider the padel deal. Difference between counted applications and actual applications is large. We see that value counts reveals the real application count (should be 148)

In [ ]:
padel_deal_id = more_rows_than_apps.loc[44, 'deal_id']
df_padel_deal = df_applications[df_applications['deal_id'] == padel_deal_id]


In [128]:
df_padel_deal.updated_at.value_counts()

updated_at
2025-10-28 11:43:58.598926    148
2025-05-01 04:17:15.752222      1
2025-05-01 04:17:15.759072      1
2025-05-01 04:17:15.833237      1
2025-05-01 04:17:15.885660      1
2025-05-01 04:17:15.903333      1
2025-05-01 04:17:15.990058      1
2025-05-01 04:17:15.995716      1
2025-05-01 04:17:16.048778      1
2025-05-01 04:17:16.087893      1
Name: count, dtype: int64

In [ ]:
more_rows_than_apps['diff_A'] = df['applicants_applications_count'] - df['nrow_in_deal_applications_table']
more_rows_than_apps['diff_B'] = df['applicants_applications_count'] - df['nrow_in_deal_applications_table_without_cancelled_status']

/var/folders/j6/_nwsrmhx5w1fj_nrxmrss8180000gq/T/ipykernel_86023/1053291677.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  more_rows_than_apps['diff_A'] = df['applicants_applications_count'] - df['nrow_in_deal_applications_table']
/var/folders/j6/_nwsrmhx5w1fj_nrxmrss8180000gq/T/ipykernel_86023/1053291677.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  more_rows_than_apps['diff_B'] = df['applicants_applications_count'] - df['nrow_in_deal_applications_table_without_cancelled_status']


We create new_df, which contains all deals where the counted nr of applications does not correspond to real number of applications. We sort by descending, so largest number of counted applications comes first. 

In [129]:
new_df = df[df['nrow_in_deal_applications_table'] != df['applicants_applications_count']].sort_values('nrow_in_deal_applications_table', ascending= False).reset_index()

In [73]:
new_df

,index,applicants_applications_count,content_types,deal_id,main_image,min_social_media_followers,deal_tags,created_at,updated_at,deleted_at,...,social_requirement_type_id,product_name,schedule_type,schedule_model,legacy_id,tags,gender,featured_image,company_id,nrow_in_deal_applications_table
0,2983,200,"[{'id': 23, 'name': 'Food', 'slug': 'Pizza'}]",019689e2-659b-00c8-90af-7978ca963643,uploads/deals/019689e2-6642-ffff-02bb-57d564ba...,5000,None,2023-09-13 08:31:07.545077,2025-05-01 03:27:18.317664,NaT,...,3.0,,online_indefinitely_schedule,{},109.0,None,None,None,None,1105
1,1513,33,"[{'id': 20, 'name': 'Beauty', 'slug': 'Heart'}]",019689e5-04f5-00c8-c7d9-11658b77f256,uploads/deals/019689e5-0553-ffff-8305-c9555e3c...,5000,None,2024-03-20 15:58:14.407003,2025-05-01 03:30:10.115138,NaT,...,3.0,,online_indefinitely_schedule,{},839.0,None,None,None,None,1089
2,2131,2,"[{'id': 21, 'name': 'Fashion', 'slug': 'Dress'}]",019689e2-814a-00c8-03b5-b326fa101bd4,uploads/deals/019689e2-84d6-ffff-d06f-da8695fa...,2500,None,2024-01-05 15:02:13.096901,2025-05-01 03:27:26.197093,2024-06-18 11:58:31.371364+00:00,...,2.0,,online_indefinitely_schedule,{},563.0,None,None,None,None,1087
3,2844,40,"[{'id': 20, 'name': 'Beauty', 'slug': 'Heart'}]",019689e2-af4e-00c8-a0ea-dea1bcf616d8,uploads/deals/019689e2-afd0-ffff-f254-751f568a...,5000,None,2024-01-16 14:44:13.894808,2025-05-01 03:27:37.937024,2024-10-24 09:26:57.138695+00:00,...,3.0,,online_indefinitely_schedule,{},581.0,None,None,None,None,1085
4,1685,531,"[{'id': 23, 'name': 'Food', 'slug': 'Pizza'}]",019689e4-da92-00c8-8339-34136a24f15a,uploads/deals/019689e4-dc5e-ffff-914e-6c6088a5...,2500,None,2023-10-25 11:29:55.311449,2025-10-28 10:55:52.953410,NaT,...,2.0,,physical_every_day_all_day_schedule,"{'indefinitely': True, 'date_schedule_end_at':...",373.0,None,None,None,0199f1d9-59da-015e-8ac9-bf4ea893b9d0,1060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,4602,0,"[{'id': 41, 'name': 'Activities', 'slug': 'per...",019689e3-e715-00c8-1ede-7eb4c884eb3f,uploads/deals/019689e3-e803-ffff-0011-2b7dae8f...,2500,None,2025-04-17 11:35:01.437143,2025-10-28 10:49:50.055409,NaT,...,2.0,,physical_specific_days_all_day_schedule,"{'indefinitely': True, 'date_schedule_end_at':...",4237.0,None,None,None,0199f19f-f708-015e-b57c-0760ec607e8c,1
954,3542,0,"[{'id': 41, 'name': 'Activities', 'slug': 'per...",019689e4-2cff-00c8-ce1c-a572502e6d4e,uploads/deals/019689e4-2de8-ffff-7063-514e3616...,10000,None,2024-12-20 14:00:13.387238,2025-05-01 03:29:14.910126,2025-01-31 08:45:58.238355+00:00,...,4.0,,online_indefinitely_schedule,{},2585.0,None,None,None,None,1
955,1966,0,"[{'id': 20, 'name': 'Beauty', 'slug': 'Heart'}...",019689e4-fe12-00c8-53e1-f1acdf63900d,uploads/deals/019689e4-ff10-ffff-f371-7fbc4ee4...,5000,None,2024-07-10 11:48:27.600162,2025-05-01 03:30:08.476722,2024-10-24 09:08:26.635132+00:00,...,3.0,,online_indefinitely_schedule,{},1277.0,None,None,None,None,1
956,3292,0,"[{'id': 29, 'name': 'UGC', 'slug': 'Film-Strip...",019689e2-e167-00c8-45d0-ba89a3d956aa,uploads/deals/019689e2-e287-ffff-4bdd-9b2130b7...,5000,None,2024-09-16 20:34:13.047614,2025-05-01 03:27:50.177764,2024-10-14 09:45:12.000028+00:00,...,3.0,,online_indefinitely_schedule,{},1657.0,None,None,None,None,1


Let's inspect the Ochama deal, which has 200 applicants, but a total of 1105 rows in df_applicants. Again, we see that 28-10 seems to be the magic date. This signals data quality issues, but we can still assume that created_at is reliable, which we have verified: the created_at dates are all unique. 

In [130]:
ochama_deal_id = new_df.reset_index().loc[0, 'deal_id']

In [131]:
df_ochama = df_applications[df_applications.deal_id == ochama_deal_id]

In [136]:
df_ochama.updated_at.value_counts()

updated_at
2025-10-28 11:44:03.476403    200
2025-05-01 04:14:22.602000      1
2025-05-01 04:14:22.615007      1
2025-05-01 04:14:22.623157      1
2025-05-01 04:14:22.631557      1
                             ... 
2025-05-01 04:14:13.765945      1
2025-05-01 04:14:13.773372      1
2025-05-01 04:14:13.779027      1
2025-05-01 04:14:13.784871      1
2025-05-01 04:14:35.697238      1
Name: count, Length: 906, dtype: int64

In [ ]:
def apply_value_counts(row):
    subs = df_applications[df_applications['deal_id'] == row['deal_id']]
    try:
        return subs.updated_at.value_counts().iloc[0]
    except:
        return None

In [140]:
b = more_rows_than_apps.apply(apply_value_counts, axis = 1)

In [143]:
more_rows_than_apps['index_0_value_counts'] = b
diff = more_rows_than_apps['applicants_applications_count'] - b
more_rows_than_apps['diff'] = diff


/var/folders/j6/_nwsrmhx5w1fj_nrxmrss8180000gq/T/ipykernel_86023/1825840282.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  more_rows_than_apps['index_0_value_counts'] = b
/var/folders/j6/_nwsrmhx5w1fj_nrxmrss8180000gq/T/ipykernel_86023/1825840282.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  more_rows_than_apps['diff'] = diff


In [ ]:
# import uuid
# df_deals_log['deal_id'] = df_deals_log.apply(lambda x: uuid.UUID(x['metadata']['entity_global_id']), axis = 1)

In [100]:
len(df_ochama) - sum(df_ochama['status'] == 'cancelled')

168

In [101]:
df_ochama[df_ochama['status'] != 'cancelled']

,id,partner_id,location_id,deal_id,creator_id,rescheduling_deal_application_id,schedule_date,status,cancelled_at,cancelled_by,cancelled_by_user_id,cancellation_reason,created_at,updated_at,deleted_at,legacy_id,cancellation_remarks
32826,019689fa-7c5a-00c9-c258-dae9e96ac8ca,6,23.0,019689e2-659b-00c8-90af-7978ca963643,3401,None,2024-02-27 00:00:00+00:00,accepted,NaT,None,NaN,None,2024-01-31 00:59:31.397184,2025-10-28 11:44:03.476403,NaT,18899.0,[other]
32829,019689fa-885e-00c9-70be-e5cb8c3cb109,6,23.0,019689e2-659b-00c8-90af-7978ca963643,3038,None,2024-03-09 00:00:00+00:00,accepted,NaT,None,NaN,None,2024-02-07 10:17:24.319519,2025-10-28 11:44:03.476403,NaT,20706.0,[other]
32842,019689fa-9116-00c9-8799-d977b32d7fe8,6,23.0,019689e2-659b-00c8-90af-7978ca963643,5946,None,2024-03-03 00:00:00+00:00,accepted,NaT,None,NaN,None,2024-02-27 15:08:53.220171,2025-10-28 11:44:03.476403,NaT,24956.0,[other]
32843,019689fa-7b3c-00c9-66f6-89c014fc4dba,6,23.0,019689e2-659b-00c8-90af-7978ca963643,982,None,2024-01-31 00:00:00+00:00,rejected,NaT,None,NaN,None,2024-01-30 14:47:50.070862,2025-10-28 11:44:03.476403,NaT,18791.0,[other]
32976,019689fa-8b33-00c9-8fc1-85c9298163d8,6,23.0,019689e2-659b-00c8-90af-7978ca963643,1920,None,2024-03-02 00:00:00+00:00,accepted,NaT,None,NaN,None,2024-02-12 20:49:59.785474,2025-10-28 11:44:03.476403,NaT,21796.0,[other]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119809,019689fa-54c7-00c9-1456-0237ea889378,6,23.0,019689e2-659b-00c8-90af-7978ca963643,4306,None,2024-01-05 00:00:00+00:00,finished,NaT,None,NaN,None,2024-01-02 14:30:52.685437,2025-10-28 11:44:03.476403,NaT,13783.0,[other]
119817,019689fa-5402-00c9-e2f9-431fdde4e000,6,23.0,019689e2-659b-00c8-90af-7978ca963643,4071,None,2024-01-05 00:00:00+00:00,finished,NaT,None,NaN,None,2023-12-29 11:01:08.114250,2025-10-28 11:44:03.476403,NaT,13255.0,[other]
119819,019689fa-4e82-00c9-57e7-7ab68c12d573,6,23.0,019689e2-659b-00c8-90af-7978ca963643,3836,None,2023-12-27 00:00:00+00:00,finished,NaT,None,NaN,None,2023-12-22 10:49:24.287603,2025-10-28 11:44:03.476403,NaT,12530.0,[other]
119839,019689fa-4732-00c9-04fa-3b0bdacc6fc7,6,23.0,019689e2-659b-00c8-90af-7978ca963643,3043,None,2023-12-14 00:00:00+00:00,finished,NaT,None,NaN,None,2023-12-14 09:18:33.401507,2025-10-28 11:44:03.476403,NaT,10323.0,[other]


In [95]:
df_deals_log[df_deals_log['deal_id'] == ochama_deal_id]

,created_at,updated_at,deleted_at,id,entity_type,entity_id,action,before,after,metadata,deal_id
50109,2025-05-01 03:27:18.311089,2025-07-11 14:40:03.608511,None,019689e2-6727-00cd-3b68-627d37142556,deal,019689e2-659b-00c8-90af-7978ca963643,DEAL_MIGRATED,"{'any': {'CalendarType': '', 'DealType': 'Onli...","{'deal': {'SoftDeletable': {}, 'description': ...",{'entity_global_id': '019689e2-659b-00c8-90af-...,019689e2-659b-00c8-90af-7978ca963643


In [89]:
df_ochama.creator_id.value_counts()

creator_id
250      26
3494     25
2268     22
5780     18
5827     18
         ..
14382     1
2663      1
4973      1
4949      1
3836      1
Name: count, Length: 395, dtype: int64

# McGill Q&A dataset

In [4]:
from datasets import load_dataset

data_files = {
    "train": "https://huggingface.co/datasets/McGill-NLP/feedbackQA/resolve/main/data/feedback_train.json",
    "validation": "https://huggingface.co/datasets/McGill-NLP/feedbackQA/resolve/main/data/feedback_valid.json",
    "test": "https://huggingface.co/datasets/McGill-NLP/feedbackQA/resolve/main/data/feedback_test.json"
}

ds = load_dataset("json", data_files=data_files)
ratings = pd.DataFrame(ds["train"])


# ratings = load_dataset("McGill-NLP/feedbackQA")["train"]
# ratings = pd.DataFrame(ratings)

ratings["review_1"] = ratings["rating"].apply(lambda x: x[0])
ratings["explanation_1"] = ratings["feedback"].apply(lambda x: x[0])
ratings["review_2"] = ratings["rating"].apply(lambda x: x[1])
ratings["explanation_2"] = ratings["feedback"].apply(lambda x: x[1])
# ratings = ratings.drop(columns=["feedback"])

# Map scores to numeric values
conversion_dict = {"Excellent": 4, "Acceptable": 3, "Could be Improved": 2, "Bad": 1}
ratings["score_1"] = ratings["review_1"].map(conversion_dict)
ratings["score_2"] = ratings["review_2"].map(conversion_dict)


ratings['answer'] = ratings['passage'].apply(lambda x: x['reference']['section_content'])

id_columns = ['question', 'answer']
ratings['input_id'] = ratings.apply(lambda row: generate_content_id(row, id_columns), axis=1)

ratings.groupby("score_1").first()

,question,passage,feedback,rating,domain,review_1,explanation_1,review_2,explanation_2,score_2,answer,input_id
score_1,,,,,,,,,,,,
1,How do I get help finding a job?,"{'passage_id': 126, 'reference': {'page_title'...",[Talks about tax credits for businesses that h...,"[Bad, Acceptable]",Australia,Bad,Talks about tax credits for businesses that hi...,Acceptable,"This answer discusses the Employment Fund, whi...",3,To further assist job seekers to prepare for a...,31effc925bc04105
2,If I am in Australia on a worker holiday marke...,"{'passage_id': 581, 'reference': {'page_title'...","[Answer is about Working Holiday Makers, but d...","[Could be Improved, Acceptable]",Australia,Could be Improved,"Answer is about Working Holiday Makers, but do...",Acceptable,Answer is rather cut and dry but is also a lit...,3,No. Existing arrangements for specified work w...,610d0764ed04054d
3,Do immigration detention centers have proper PPE?,"{'passage_id': 220, 'reference': {'page_title'...",[Claims it monitors PPE levels in the immigran...,"[Acceptable, Acceptable]",Australia,Acceptable,Claims it monitors PPE levels in the immigrant...,Acceptable,This gives a general answer to the question bu...,3,PPE levels are being closely monitored to ensu...,9e1cc5d5ff9d112e
4,How do I get help finding a job?,"{'passage_id': 140, 'reference': {'page_title'...",[Has a link to detailed information about gove...,"[Excellent, Could be Improved]",Australia,Excellent,Has a link to detailed information about gover...,Could be Improved,"This answer provides a link for job searches, ...",2,If you are a current job seeker or participant...,eff9e000675931f5


In [5]:
ratings.to_csv('../data/QA_McGill_ratings.csv')